In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# import os
# # locate current file(colab) folder: you can change to your own path
# import sys
# os.chdir('/content/drive/My Drive/RR-project/NL2GraphQuery-workplace/code/NLtoGQ/')
# !pwd

/content/drive/My Drive/RR-project/NL2GraphQuery-workplace/code/NLtoGQ


In [7]:
import spacy
import pandas as pd
import os
import params
from params.configures import Config_path, Config_output_path

In [8]:
## locate parent directory
# It suggest to train models on Google Colab GPU (connect to your own google drive)
config = Config_path()
os.chdir(config.parent_path)

In [10]:
#########################
## Load trained models ##
#########################
NER = spacy.load("./oneHop_QA/NER_output_train-dev/model-best") #load the best model
CATS = spacy.load("./oneHop_QA/cat_output_train-train_268_0719/model-best") #load the best model

C:\Users\junru\anaconda3\envs\NL2GraphQuery\lib\site-packages\spacy\util.py:730: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
question = "which genre of album is harder faster"
doc = NER(question) # input sample text
print(doc.ents[0])
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter Notebook/Colab

harder faster


In [ ]:
#####################
## Display Metrics ##
#####################
from oneHop_QA.metrics import test_pipe,mask_doc,NER_metrics
predict_cats = test_pipe(question,NER,CATS)
print('relation prediction:',predict_cats)

In [ ]:
c = Config_path()
test = pd.read_csv(c.test_combination_path)
test_q = test['question'].values
test_ents = test['entity'].values
precision, recall, f1= NER_metrics(test_q[0:1000],test_ents[0:1000],NER)
print('Named Entity Metrics',precision, recall, f1)

In [ ]:
from sklearn.metrics import f1_score

test_q_space = test['q_space'].values
test_r = test['relationship'].values
test_r = [i.rsplit('/', 2)[0:-2] for i in test_r]

redict_r = []
for i in test_q_space[0:1000]:
  doc_cats = CATS(i)
  pedict_relation = max(doc_cats.cats, key=doc_cats.cats.get)
  redict_r.append(pedict_relation)

In [ ]:
f1_CATS = f1_score(test_r[0:1000], redict_r, average='weighted')
print('relation prediction f1 score:',f1_CATS)